In [2]:
%load_ext autoreload
%autoreload 2

from mimic_iv_analysis.core.params import DEFAULT_STUDY_TABLES_LIST
from mimic_iv_analysis.io import DataLoader, TableNamesHOSP, TableNamesICU, ParquetConverter, ExampleDataLoader
import pandas as pd

import dask.dataframe as dd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# from dask.distributed import Client
# client = Client(n_workers=4, threads_per_worker=4, processes=True, memory_limit='8GB')
# client
[a.value for a in DEFAULT_STUDY_TABLES_LIST]

['patients',
 'admissions',
 'diagnoses_icd',
 'transfers',
 'd_icd_diagnoses',
 'poe',
 'poe_detail']

## Scan the directory

In [ ]:
len( list(set.intersection(*a[:4])) )

In [ ]:
# TODO: Load tables individually and merge them with the how=outer (filters applied after merging) and see if it makes any difference compare to when I do it with the inner (filters applied before merging).

# examples_full    = ExampleDataLoader(partial_loading=False)
examples_partial = ExampleDataLoader(partial_loading=False, num_subjects=10, random_selection=False)

In [ ]:
examples_partial.study_table_info()

In [ ]:
examples_partial.counter()

In [ ]:
df = examples_partial.load_merged_tables(partial_loading=True, num_subjects=10, random_selection=False, use_dask=True)

In [ ]:
# Print in a tabular format
examples_partial.n_rows_after_merge()